In [3]:
import numpy as np
import pandas as pd
from sklearn import preprocessing
from sklearn import model_selection
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier


In [4]:
training=pd.read_csv('train.csv').copy()
testing=pd.read_csv('test.csv').copy()

In [5]:
def count_nulls(dataset):
    return sum(dataset.isnull())

In [6]:
def get_name(my_list, position):
    return my_list[position][:4]

In [7]:
training['Updated_Names']=training['Name'].str.split(',').apply(get_name, position=1)
training.drop('Name', axis=1, inplace=True)

In [8]:
d=dict()
count=0
for i in set(training['Updated_Names']):
    d[i]=count/15
    count+=1
training['Names']=training['Updated_Names'].map(d)
training.drop('Updated_Names', axis=1, inplace=True)

In [9]:
training['sex']=training['Sex'].map({"female":0, "male":1})
training.drop('Sex', axis=1, inplace=True)

In [10]:
training['pclass']=training['Pclass']/3
training.drop('Pclass', axis=1, inplace=True)

In [11]:
training.loc[training['Embarked'].isnull(), 'Embarked']='S'

In [12]:
training['embarked']=training.Embarked.map({"S":0, "C":0.5, "Q":1})
training.drop('Embarked', inplace=True, axis=1)

In [13]:
training['fare']=training.Fare/training.Fare.max()
training.drop('Fare', inplace=True, axis=1)

In [14]:
training['parch']=training['Parch']/training['Parch'].max()
training.drop('Parch', inplace=True, axis=1)

In [15]:
training['sibsp']=training['SibSp']/training['SibSp'].max()
training.drop('SibSp', inplace=True, axis=1)

In [16]:
cabins=[]
for i in training.loc[training['Cabin'].notnull(), 'Cabin']:
    cabins.append(i[0])

In [17]:
set_cabins=set(cabins)

In [18]:
dictionary_cabin=dict()
count=0
for i in set_cabins:
    dictionary_cabin[i]=count/8
    count+=1
dictionary_cabin['N']=1
dictionary_cabin

{'E': 0.0,
 'C': 0.125,
 'G': 0.25,
 'A': 0.375,
 'F': 0.5,
 'D': 0.625,
 'T': 0.75,
 'B': 0.875,
 'N': 1}

In [19]:
training.loc[training.Cabin.isnull(), 'Cabin']='N'

In [20]:
def cabin_mapping(get_list, position):
    return get_list[position][0]
training['cabin']=training.Cabin.str.split(' ').apply(cabin_mapping, position=0)

In [21]:
training.drop('Cabin', axis=1, inplace=True)

In [22]:
training['cabins']=training.cabin.map(dictionary_cabin)
training.drop('cabin', inplace=True, axis=1)

In [23]:
def ticket_mapping(get_ticket, position):
    if get_ticket[position]=='LINE':
        return -1
    return int(get_ticket[position])
training['ticket']=training['Ticket'].str.split(' ').apply(ticket_mapping, position=-1)
training.drop('Ticket', inplace=True, axis=1)

In [24]:
training['tickets']=training['ticket']/training.ticket.max()
training.drop('ticket', inplace=True, axis=1)

In [25]:
training.loc[(training.Age.isnull())&(training['Survived']==0), 'Age']=training.loc[(training.Age.notnull())&(training['Survived']==0), 'Age'].median()

In [26]:
training.loc[(training.Age.isnull())&(training['Survived']==1), 'Age']=training.loc[(training.Age.notnull())&(training['Survived']==1), 'Age'].median()

In [27]:
training['age']=training.Age/80.0
training.drop('Age', inplace=True, axis=1)

In [28]:
training

,Survived,Names,sex,pclass,embarked,fare,parch,sibsp,cabins,tickets,age
0,1,0.400000,0,0.666667,0.0,0.050749,0.000000,0.125,1.0,0.073651,0.3625
1,0,0.066667,1,1.000000,0.0,0.015713,0.000000,0.000,1.0,0.000795,0.3625
2,0,0.066667,1,0.666667,0.0,0.050749,0.000000,0.000,1.0,0.080822,0.4875
3,0,0.400000,0,1.000000,0.0,0.041136,0.666667,0.000,1.0,0.112826,0.3625
4,0,0.066667,1,1.000000,0.0,0.013761,0.000000,0.000,1.0,0.126422,0.3125
...,...,...,...,...,...,...,...,...,...,...,...
663,1,0.333333,0,0.666667,0.0,0.020495,0.000000,0.000,1.0,0.004800,0.2125
664,0,0.066667,1,1.000000,1.0,0.015127,0.000000,0.000,1.0,0.120150,0.3625
665,1,0.066667,1,1.000000,0.0,0.110272,0.000000,0.000,1.0,0.000516,0.4000
666,0,0.333333,0,1.000000,0.0,0.019202,0.000000,0.000,1.0,0.002435,0.2750


In [29]:
values=training.values
from sklearn.model_selection import train_test_split
X=values[:, 1:]
Y=values[:, 0]
x_train, x_test, y_train, y_test=train_test_split(X, Y)

In [30]:
clf=LogisticRegression(n_jobs=-1, solver="saga", max_iter=20000, C=10, tol=0.000001, class_weight="balanced")
clf.fit(x_train, y_train)
clf.score(x_test, y_test)

0.8083832335329342

In [31]:
clf

LogisticRegression(C=10, class_weight='balanced', max_iter=20000, n_jobs=-1,
                   solver='saga', tol=1e-06)